In [1]:
import copy
import numpy as np
from tqdm import tqdm
import warnings
from numpy.random import normal, randint
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
import re
%load_ext autoreload
%autoreload 2

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


from inn import *
from datasetall.datautils import *
from clfs.clfutils import *

# load dataset utils
from carla.data.catalog import CsvCatalog
continuous_features = ["duration", "amount", "age"]
columns = ["checking-status", "duration", "credit-history", "purpose", "amount", "savings", "employment", "rate",
           "sex-status", "guarantors",
           "residence", "property", "age", "installment", "housing", "num-credits", "job", "liable", "phone", "foreign",
           "good-credit"]
# load dataset manually
from carla.data.catalog import CsvCatalog
dataset_csv = CsvCatalog(file_path="../datasetall/credit/old/german_processed.csv", categorical=[i for i in columns if i not in continuous_features and i != "good-credit"], continuous=continuous_features, immutables=["sex-status"], target="good-credit")
# load dataset utils
dataset = InnDataSet(name="german", carla_dataset=dataset_csv)
X, y, df, columns, ordinal_features, discrete_features, continuous_features, feat_var_map = load_dataset_utils(
    dataset)
dataset_name = "credit"

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.
 [lazy_loader.py _load]


In [2]:
print(X.shape, len(discrete_features))

(1150, 20) 17


In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
# these X test is for testing multiplicity
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
seed = 0
clf0 = MLPClassifier(hidden_layer_sizes=(20, 20), random_state=seed, learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)
clf0.fit(X_train, y_train)

MLPClassifier(batch_size=64, hidden_layer_sizes=(20, 20),
              learning_rate='adaptive', learning_rate_init=0.02,
              random_state=0)

In [3]:
clf = MLPClassifier(hidden_layer_sizes=(15, 15), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.7087 test_accuracy with a std of 0.0287
0.6924 test_f1_macro with a std of 0.0291


In [4]:
clf = MLPClassifier(hidden_layer_sizes=(20, 15), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.6989 test_accuracy with a std of 0.0074
0.6836 test_f1_macro with a std of 0.0067


In [5]:
clf = MLPClassifier(hidden_layer_sizes=(20, 20), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.7054 test_accuracy with a std of 0.0183
0.6897 test_f1_macro with a std of 0.0170


In [6]:
clf = MLPClassifier(hidden_layer_sizes=(30, 20), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.6967 test_accuracy with a std of 0.0116
0.6846 test_f1_macro with a std of 0.0141


In [7]:
clf = MLPClassifier(hidden_layer_sizes=(30, 25), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.6957 test_accuracy with a std of 0.0203
0.6794 test_f1_macro with a std of 0.0238


In [8]:
clfs = []
total_num = 150
seed = 0
scores_acc = []
scores_compl = []
sizes = [(15,15),(20,15),(20,20),(30,20),(30,25)]
sizes_score = [0, 0.25, 0.5, 0.75, 1]
for i in tqdm(range(0, total_num)):
    X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_train, y_train, test_size=0.2, random_state=2*seed+15)
    sizes_idx = np.random.randint(0,5)
    this_clf = MLPClassifier(hidden_layer_sizes=sizes[sizes_idx], random_state=3*seed+21, learning_rate='adaptive', batch_size=64, learning_rate_init=0.02).fit(X_train_0, y_train_0)
    seed += 12
    if len(np.unique(this_clf.predict(X_test))) == 1:
        continue
    scores_acc.append(accuracy_score(this_clf.predict(X_test), y_test))
    scores_compl.append(sizes_score[sizes_idx])
    #if accuracy_score(y_test, this_clf.predict(X_test)) < 0.73 or accuracy_score(y_test, this_clf.predict(X_test)) > 0.77:
    #    continue
    clfs.append(this_clf)
clfs.append(clf0)
print(len(clfs))

100%|██████████| 150/150 [00:29<00:00,  5.05it/s]

151


In [9]:
from methods import Datasets, ExperimentsForOneModelSize
d_idxs = np.arange(0, len(X_test))
if len(X_test) >= 1000:
    d_idxs = np.random.randint(0, len(X_test) - 1, 1000)
X_test_run = pd.DataFrame(data=X_test.values[d_idxs], columns=X_test.columns)
y_test_run = pd.DataFrame(data=y_test.values[d_idxs], columns=y_test.columns)

d = Datasets(X_train, y_train, X_test_run, y_test_run)

In [10]:
exp_10 = ExperimentsForOneModelSize(d, clfs, scores_acc, scores_compl, model_size=10, dataset_name="credit", seed=3333)
exp_10.eval()

100%|██████████| 5/5 [00:31<00:00,  6.30s/it]


In [11]:
exp_10.eval_ours()
exp_10.print_results()

100%|██████████| 5/5 [00:03<00:00,  1.30it/s]

model size = 10, mean, single model acc: 0.6643478260869566
+---------+-------+----------+---------+---------+---------+-----------+-------------+--------------------+--------------+
| name    |   acc |   m simp |   m len |   c len |   c val |   abstain |   multi_ans |   multi_same_class |   same_as_mv |
+=========+=======+==========+=========+=========+=========+===========+=============+====================+==============+
| mvca    | 0.697 |    0.588 |   0.817 |   0.817 |   0.757 |     0.000 |       0.074 |                    |              |
| mvcva   | 0.697 |    0.588 |   0.817 |   0.457 |   1.000 |     0.436 |       0.074 |                    |              |
| ours    | 0.694 |    0.600 |   0.580 |   0.580 |   1.000 |     0.000 |       0.100 |              0.046 |        0.953 |
| ours-a  | 0.710 |    0.593 |   0.486 |   0.486 |   1.000 |     0.000 |       0.015 |              0.010 |        0.825 |
| ours-s  | 0.689 |    0.626 |   0.565 |   0.565 |   1.000 |     0.000 |       

In [12]:
exp_20 = ExperimentsForOneModelSize(d, clfs, scores_acc, scores_compl, model_size=20, dataset_name="credit", seed=3333)
exp_20.eval()

100%|██████████| 5/5 [01:04<00:00, 12.83s/it]


In [13]:
exp_20.eval_ours()
exp_20.print_results()

100%|██████████| 5/5 [00:06<00:00,  1.27s/it]

model size = 20, mean, single model acc: 0.6626521739130435
+---------+-------+----------+---------+---------+---------+-----------+-------------+--------------------+--------------+
| name    |   acc |   m simp |   m len |   c len |   c val |   abstain |   multi_ans |   multi_same_class |   same_as_mv |
+=========+=======+==========+=========+=========+=========+===========+=============+====================+==============+
| mvca    | 0.708 |    0.571 |   0.810 |   0.810 |   0.734 |     0.000 |       0.033 |                    |              |
| mvcva   | 0.708 |    0.571 |   0.810 |   0.351 |   1.000 |     0.618 |       0.033 |                    |              |
| ours    | 0.691 |    0.580 |   0.557 |   0.557 |   1.000 |     0.000 |       0.057 |              0.023 |        0.961 |
| ours-a  | 0.709 |    0.586 |   0.481 |   0.481 |   1.000 |     0.000 |       0.014 |              0.007 |        0.862 |
| ours-s  | 0.684 |    0.590 |   0.549 |   0.549 |   1.000 |     0.000 |       

In [14]:
exp_30 = ExperimentsForOneModelSize(d, clfs, scores_acc, scores_compl, model_size=30, dataset_name="credit", seed=3333)
exp_30.eval()

100%|██████████| 5/5 [01:58<00:00, 23.61s/it]


In [15]:
exp_30.eval_ours()
exp_30.print_results()

100%|██████████| 5/5 [00:10<00:00,  2.17s/it]

model size = 30, mean, single model acc: 0.6628115942028987
+---------+-------+----------+---------+---------+---------+-----------+-------------+--------------------+--------------+
| name    |   acc |   m simp |   m len |   c len |   c val |   abstain |   multi_ans |   multi_same_class |   same_as_mv |
+=========+=======+==========+=========+=========+=========+===========+=============+====================+==============+
| mvca    | 0.710 |    0.540 |   0.807 |   0.807 |   0.727 |     0.000 |       0.022 |                    |              |
| mvcva   | 0.710 |    0.540 |   0.807 |   0.311 |   1.000 |     0.699 |       0.022 |                    |              |
| ours    | 0.683 |    0.548 |   0.551 |   0.551 |   1.000 |     0.000 |       0.038 |              0.019 |        0.943 |
| ours-a  | 0.711 |    0.552 |   0.441 |   0.441 |   1.000 |     0.000 |       0.010 |              0.005 |        0.850 |
| ours-s  | 0.684 |    0.555 |   0.546 |   0.546 |   1.000 |     0.000 |       